## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из яндекс-диска

In [1]:
import os
import pandas as pd
path = r'/Users/kirill/Downloads/Activities-analysis-main/kaggle_data' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("/")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("/")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("/")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("/")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("/")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("/")[-1])
                pass

track_0.csv
track_1.csv
track_10.csv
track_100.csv
track_101.csv
track_102.csv
track_103.csv
track_104.csv
track_105.csv
track_106.csv
track_107.csv
track_108.csv
track_109.csv
track_11.csv
track_110.csv
track_111.csv
track_112.csv
track_113.csv
track_114.csv
track_115.csv
track_116.csv
track_117.csv
track_118.csv
track_119.csv
track_12.csv
track_120.csv
track_121.csv
track_122.csv
track_123.csv
track_124.csv
track_125.csv
track_126.csv
track_127.csv
track_128.csv
track_129.csv
track_13.csv
track_130.csv
track_131.csv
track_132.csv
track_133.csv
track_134.csv
track_135.csv
track_136.csv
track_137.csv
track_138.csv
track_139.csv
track_14.csv
track_140.csv
track_141.csv
track_142.csv
track_143.csv
track_144.csv
track_145.csv
track_15.csv
track_16.csv
track_17.csv
track_18.csv
track_19.csv
track_2.csv
track_20.csv
track_21.csv
track_22.csv
track_23.csv
track_24.csv
track_25.csv
track_26.csv
track_27.csv
track_28.csv
track_29.csv
track_3.csv
track_30.csv
track_31.csv
track_32.csv
track_33.

Переведём типы движений в числа

In [2]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [3]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [4]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / numpy.linalg.norm(vec1)).reshape(3), (vec2 / numpy.linalg.norm(vec2)).reshape(3)
    v = numpy.cross(a, b)
    if any(v): #if not all zeros then 
        c = numpy.dot(a, b)
        s = numpy.linalg.norm(v)
        kmat = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return numpy.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3) #cross of all zeros only occurs on identical directions

In [5]:
def rotate(X, Y, Z, Mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(X,Y,Z):
        res = Mat.dot(elem)
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

In [6]:
def integrate(lst):
    res = []
    s = 0
    for elem in lst:
        s += elem
        res.append(s)
    return res

In [20]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):
    
# обрезаем начало и конец трека
    if a == -1:
        st = 1250
    else:
        st = a
            
    if b == -1:
        end = -1250
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1

    print(name)
#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return [0, 0, 0, 0, 0]

######
# integration
######

    mx = df.iloc[st:end]['gFx'].values.mean()
    my = df.iloc[st:end]['gFy'].values.mean()
    mz = df.iloc[st:end]['gFz'].values.mean()
    
    Vec = [mx,my,mz]
    Point = [0, 0, 1]
    mat = rotation_matrix_from_vectors(Vec, Point)
    
    new_x, new_y, new_z = rotate(df.iloc[st:end]['gFx'].values, df.iloc[st:end]['gFy'].values, df.iloc[st:end]['gFz'].values, mat)

    print('New means:', mean(new_x), mean(new_y), mean(new_z))
    
    is_z_cor = (1 if (abs(mean(new_x)) + abs(mean(new_y)) + abs(mean(new_x)) - 1) < 3 else 0)
    
    new_z = [(elem-1)/62500 for elem in new_z]
    
    int_z = integrate(integrate(new_z))
    try:
        Ampl_z = max(int_z)-min(int_z)
    except ValueError:
        return [0, 0, 0, 0, 0]
    print("Ampl Z:", Ampl_z)
    
######
######

#####
# Furye
#####

    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

    Fs = 250
    y = Acc.values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    
    furye_max = yabs[1:].max()
    furye_maxarg = yabs[1:].argmax()
    print("Furye:", furye_max)

####
####

# рисуем графики ускорений
#     if cut:
#         ax1.plot(df.iloc[st:end]['gFx'],c='g')
#         ax1.plot(df.iloc[st:end]['gFy'],c='r')
#         ax1.plot(df.iloc[st:end]['gFz'],c='b')
#     else:
#         ax1.plot(df['gFx'],c='g')
#         ax1.plot(df['gFy'],c='r')
#         ax1.plot(df['gFz'],c='b')
        
# вычисляем модуль (норму) ускорения и рисуем его
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
#     ax2.plot(Acc)
#     plt.show()

# можно различить многие действия по амплитуде ускорения
    blue_ampl = df.gFz[st:end].max() - df.gFz[st:end].min()
    amplitude = Acc.max() - Acc.min()
    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
    print()
    
    return [amplitude, furye_max, Ampl_z, blue_ampl, furye_maxarg]

#Ampl = np.zeros(len(Data))
Ampl = [0] * len(Data)

for df, target in zip(Data,Target):
    #print("gfd",target.split('_')[1].split('.')[0], "Gfdgf")
    track_num = int(target.split('_')[1].split('.')[0]) # важный момент, что треки не по нумерации идут!
    Ampl[track_num] = process_data(df, name=target)

Populating the interactive namespace from numpy and matplotlib
track_0.csv
New means: 5.130911189502715e-17 -2.3000636366736312e-17 0.9882241021345644
Ampl Z: 0.07224694754223372
Furye: 0.1238286899027346
 АМПЛИТУДА: 1.2409331302405158

track_1.csv
New means: -1.107534466531484e-16 4.3622801745222565e-18 1.0093063544485352
Ampl Z:

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


 62.61452770730861
Furye: 0.0759017653826047
 АМПЛИТУДА: 1.9935598883036225

track_10.csv
New means: -2.4209292530156736e-17 7.26278775904702e-17 1.0629038381437543
Ampl Z: 0.9457827346098793
Furye: 0.5960586018001915
 АМПЛИТУДА: 7.802304392990605

track_100.csv
New means: -1.8042329267571705e-18 5.279102907068881e-17 1.027486206176945
Ampl Z: 57.053470131057075
Furye: 0.00017314348329196413
 АМПЛИТУДА: 0.34113272676839523

track_101.csv
New means: -1.3563673712355336e-17 4.6504024156646864e-17 1.0084850836181023
Ampl Z: 1.0683705088237156
Furye: 0.006912791043251188
 АМПЛИТУДА: 0.4539765748076483

track_102.csv
New means: 0.0 -1.179324042755353e-18 0.9909631896954836
Ampl Z: 9.704615848731523
Furye: 0.10420805990631737
 АМПЛИТУДА: 2.2110203625988167

track_103.csv
New means: 3.671208488496158e-17 -3.1467501329967073e-17 0.7389066186365724
Ampl Z: 432.0261484516366
Furye: 0.12425585072809714
 АМПЛИТУДА: 3.1345378975461413

track_104.csv
New means: 7.973671598305364e-17 9.52410774242029

<ipython-input-20-d0c47a92004d>:40: RuntimeWarning: Mean of empty slice.
  mx = df.iloc[st:end]['gFx'].values.mean()
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-20-d0c47a92004d>:41: RuntimeWarning: Mean of empty slice.
  my = df.iloc[st:end]['gFy'].values.mean()
<ipython-input-20-d0c47a92004d>:42: RuntimeWarning: Mean of empty slice.
  mz = df.iloc[st:end]['gFz'].values.mean()
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


New means: -1.3263500677096372e-18 9.2957867993325e-18 1.005828654877506
Ampl Z: 17.996954404146376
Furye: 0.00033873086850907773
 АМПЛИТУДА: 0.042436878839561376

track_112.csv
New means: -1.909687767142568e-17 6.944319153245701e-19 0.9871833395618854
Ampl Z: 0.18122208543146284
Furye: 0.14987781368316921
 АМПЛИТУДА: 1.4825475120699383

track_113.csv
New means: 0.0 1.2468700213192107e-17 1.0037697191452375
Ampl Z: 13.591567418544763
Furye: 0.008832908830105885
 АМПЛИТУДА: 0.7902169184700258

track_114.csv
New means: -5.141409086541969e-18 -7.712113629812955e-18 1.003469680392359
Ampl Z: 0.027825126110717042
Furye: 0.07244769543520613
 АМПЛИТУДА: 0.8574945808044203

track_115.csv
New means: 4.035645981219804e-18 -2.017822990609902e-17 1.0211087501013512
Ampl Z: 1.1268232081254195
Furye: 0.2009515281136816
 АМПЛИТУДА: 5.0103722896175285

track_116.csv
New means: -1.3987061727561027e-17 -4.370956789862821e-17 1.016095135071031
Ampl Z: 0.32110064619766343
Furye: 0.16332288606034304
 АМПЛИ

New means: -1.0061327737856504e-16 -3.783507645154953e-17 1.0024064897310223
Ampl Z: 11.60277266985545
Furye: 0.02344632071024288
 АМПЛИТУДА: 0.741920211510277

track_27.csv
New means: -8.332212719741693e-17 -1.8437236656449704e-16 0.9953518410836716
Ampl Z: 0.008852970542004892
Furye: 0.07005714535601887
 АМПЛИТУДА: 2.010031339155569

track_28.csv
New means: -9.43833839011395e-17 -2.585283993813821e-16 0.9898306679841353
Ampl Z: 0.7822328780047537
Furye: 0.11192207779905504
 АМПЛИТУДА: 3.2323211416069624

track_29.csv
New means: 6.110318366487348e-17 1.134773410919079e-16 0.8939287239433704
Ampl Z: 117.48982095992044
Furye: 0.052095675376440984
 АМПЛИТУДА: 2.76747032033991

track_3.csv
New means: 1.4468064755296158e-16 2.503204854487748e-16 1.0871752995559891
Ampl Z: 1.709589359382349
Furye: 0.41928825714905993
 АМПЛИТУДА: 7.376276708180853

track_30.csv
New means: -3.055812064787267e-17 2.0826023528495831e-16 0.9802746327213991
Ampl Z: 4.496970482102569
Furye: 0.0005456195243915532
 

New means: 4.989182462914688e-17 -1.829366903068719e-16 0.6574141736183333
Ampl Z: 3.0553419098974754
Furye: 0.045120808964902344
 АМПЛИТУДА: 2.1336461804594

track_74.csv
New means: -1.3034854649484362e-17 6.826345845336898e-17 0.9951200156776878
Ampl Z: 0.427375655638903
Furye: 0.19721660997565085
 АМПЛИТУДА: 3.301098397972242

track_75.csv
New means: 1.4592696952115697e-17 2.5780431282071064e-16 1.1580226901824804
Ampl Z: 177.1079841217309
Furye: 0.39231486742141836
 АМПЛИТУДА: 7.158302893444175

track_76.csv
New means: 7.366073018246652e-18 -7.47939721852737e-17 0.9487734810439752
Ampl Z: 66.3470038919256
Furye: 0.3318445495315999
 АМПЛИТУДА: 9.216037535513616

track_77.csv
New means: 0.0 -1.1872822062829437e-16 0.9582430632896307
Ampl Z: 0.9544114507787035
Furye: 0.057947649734941986
 АМПЛИТУДА: 1.8330376632404348

track_78.csv
New means: 6.110318366487348e-17 1.134773410919079e-16 0.8939287239433704
Ampl Z: 117.48982095992044
Furye: 0.052095675376440984
 АМПЛИТУДА: 2.767470320339

Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [24]:
def get_prediction(a):
    ampl = a[0]
    furye = a[1]
    ampl_z = a[2]
    ampl_blue = a[3]
    fur_arg = a[4]
    if ampl == 0 and furye == 0 and ampl_z == 0 and ampl_blue == 0 and fur_arg == 0:
        return 1
    if ampl < 0.5:
        return 0 #стояние
    if ampl > 5:
        return 2 #бег
    if fur_arg > 300:
        return 7
    if furye < 0.06 and furye >= 0.02:
        return 3
    if furye < 0.02:
        if ampl_blue < 0.65:
            return 6
        if ampl_blue >= 0.65:
            return 5
    if furye >= 0.06:
        if ampl_z > 30:
            return 4
        else:
            return 1
    return 1 #ходьба

In [26]:
len(Data), len(Ampl), len(Target)

(146, 146, 146)

In [27]:
type(Data)

list

In [28]:
# Ampl_new = []

# i=0
# for elem in Ampl:
#     if not (math.isnan(elem) or elem == -1):
#         i+=1
#         Ampl_new.append(elem)
#     else:
#         Ampl_new.append(2)
# len(Ampl_new), i

Создадим таблицу, содержащую номер трека и предсказание.

In [25]:
prediction = [get_prediction(a) for a in Ampl]

Res = pd.DataFrame({'track_num' : np.arange(len(Data)), 'action' : prediction})
Res.head(10)

,track_num,action
0,0,1
1,1,4
2,2,3
3,3,2
4,4,4
5,5,1
6,6,1
7,7,2
8,8,3
9,9,0


In [26]:
Res.to_csv("submit.csv", index=False)